In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix
from keras.applications.densenet import DenseNet121
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
'''
In this section the CIFAR10 dataset is loaded and displayed
the class variables are also defined
The data is then scaled and transformed into one-hotencoding
'''
(Xtrain, ytrain), (Xtest, ytest) = cifar10.load_data()

labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
          'dog', 'frog', 'horse', 'ship', 'truck'] # labels of the dataset
W_grid = 10 # dimensions
L_grid = 10
fig, axes = plt.subplots(L_grid, W_grid, figsize = (17,17))
axes = axes.ravel() # flatten the 15 x 15 matrix into 225 array
n_train = len(Xtrain)

for i in np.arange(0, W_grid * L_grid):
    index = np.random.randint(0, n_train) # random number
    axes[i].imshow(Xtrain[index,1:]) # read and display image with the selected index
    label_index = int(ytrain[index])
    axes[i].set_title(labels[label_index], fontsize = 8)
    axes[i].axis('off')
plt.subplots_adjust(hspace=0.4)

# data prep
Xtrain = Xtrain / 255.0 # scale data
Xtest = Xtest / 255.0
ycattrain = to_categorical(ytrain, 10) # transform target variable into one-hotencoding
ycattest = to_categorical(ytest, 10)
ycattrain

In [ ]:
'''
This is the architecture of the basic CNN model
It uses RELU and the Adam optimizer
'''
INPUT_SHAPE = (32, 32, 3)
KERNEL_SIZE = (3, 3)
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same')) # convolutional layer
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2))) # pooling layer
model.add(Dropout(0.25)) # dropout layers
model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)
model.summary() # output summarised model structure
early_stop = EarlyStopping(monitor='val_loss', patience=2) #early stop

In [ ]:
'''
This is where the data is augmented using ImageDataGenerator
It is also the training section for the model
'''
batch_size = 32
data_generator = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = data_generator.flow(Xtrain, ycattrain, batch_size)
steps_per_epoch = Xtrain.shape[0] // batch_size
r = model.fit(train_generator, epochs=20,steps_per_epoch=steps_per_epoch,validation_data=(Xtest, ycattest))

In [ ]:
'''
This section evaluates the model and displays metrics like loss and confusion matrices
It then tests the model
'''

plt.figure(figsize=(12, 16))

plt.subplot(4, 2, 2)
plt.plot(r.history['accuracy'], label='accuracy')
plt.plot(r.history['val_accuracy'], label='val_accuracy')
plt.title('Accuracy Function Evolution')
plt.legend()
plt.subplot(4, 2, 3)
plt.plot(r.history['precision'], label='precision')
plt.plot(r.history['val_precision'], label='val_precision')
plt.title('Precision Function Evolution')
plt.legend()
plt.subplot(4, 2, 4)
plt.plot(r.history['recall'], label='recall')
plt.plot(r.history['val_recall'], label='val_recall')
plt.title('Recall Function Evolution')
plt.legend()
plt.subplot(4, 2, 1)
plt.plot(r.history['loss'], label='Loss')
plt.plot(r.history['val_loss'], label='val_Loss')
plt.title('Loss Function Evolution')
plt.legend()

In [ ]:
evaluation = model.evaluate(Xtest, ycattest) # evaluates the tests accuracy
print(f'Accuracy of test: {evaluation[1] * 100:.2f}%')

In [ ]:
ypred = model.predict(Xtest)
ypred = np.argmax(ypred, axis=1)
cm = confusion_matrix(ytest, ypred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels) # displays a confusion matrix of predicted labels to true labels
fig, ax = plt.subplots(figsize=(10, 10))
disp = disp.plot(xticks_rotation='vertical', ax=ax,cmap='summer')
plt.show()
print(classification_report(ytest, ypred))

In [ ]:
my_image = Xtest[100]
plt.imshow(my_image)
print(f" image 100 is {ytest[100]}") # test against deer image
pred_100 = np.argmax(model.predict(my_image.reshape(1, 32, 32, 3))) # correctly predicted as a deer
print(f"model prediction: {pred_100}")

In [ ]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
          'dog', 'frog', 'horse', 'ship', 'truck'] # labels of the dataset, class == 4

W_grid = 5 # defines the dimensions of the plot grid
L_grid = 5
fig, axes = plt.subplots(L_grid, W_grid, figsize = (17,17))
axes = axes.ravel()
n_test = len(Xtest)
for i in np.arange(0, W_grid * L_grid):
    index = np.random.randint(0, n_test)
    axes[i].imshow(Xtest[index,1:]) # read and display an image with selected index
    label_index = int(ypred[index])
    axes[i].set_title(labels[label_index], fontsize = 8)
    axes[i].axis('off')
plt.subplots_adjust(hspace=0.4) # displays random set of images with their indexes

In [ ]:
def plot_image(i, pred_arr, true_label, img):
    pred_arr, true_label, img = pred_arr, true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)
    pred_label = np.argmax(pred_arr)
    if pred_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel(f"{labels[int(pred_label)]} {100*np.max(pred_arr):2.0f}% ({labels[int(true_label)]})", 
               color=color)

def plot_value_array(i, pred_arr, true_label):
    pred_arr, true_label = pred_arr, int(true_label[i])
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), pred_arr, color="#777777")
    plt.ylim([0, 1])
    pred_label = np.argmax(pred_arr)
    thisplot[pred_label].set_color('red')
    thisplot[true_label].set_color('blue')
predictions = model.predict(Xtest)

# plot the first test images, predicted labels and true labels
# correct predictions in blue and incorrect in red
num_rows = 8
num_cols = 5
num_images = num_rows * num_cols
plt.figure(figsize=(2 * 2 * num_cols, 2 * num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2 * num_cols, 2 * i + 1)
    plot_image(i, predictions[i], ytest, Xtest)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], ytest)
plt.tight_layout()
plt.show()